# 📘 Агентные Архитектуры 3: ReAct (Reason + Act)

Добро пожаловать в третий ноутбук нашей серии. Теперь мы изучим **ReAct**, ключевую архитектуру, которая преодолевает разрыв между простым использованием инструментов и сложным, многошаговым решением проблем. ReAct расшифровывается как **Reason + Act** (Рассуждать + Действовать), и его главная инновация заключается в том, что он позволяет агенту динамически рассуждать о проблеме, действовать на основе своих рассуждений, наблюдать результат, а затем снова рассуждать.

Этот паттерн превращает агента из статического "вызывателя инструментов" в адаптивного решателя проблем. Чтобы подчеркнуть его мощь, мы сначала построим **базового агента, использующего инструменты один раз (single-shot)**, и покажем его ограничения на сложной задаче. Затем мы построим полноценного ReAct агента и продемонстрируем, как его итеративный цикл `think -> act -> observe` (думай -> действуй -> наблюдай) позволяет ему преуспеть там, где базовый агент терпит неудачу.

### Определение
Архитектура **ReAct** — это дизайн-паттерн, в котором агент чередует шаги рассуждения с действиями. Вместо того чтобы планировать все свои шаги заранее, агент генерирует мысль о своем непосредственном следующем шаге, выполняет действие (например, вызывает инструмент), наблюдает результат, а затем использует эту новую информацию для генерации своей следующей мысли и действия. Это создает динамичный и адаптивный цикл.

### Высокоуровневый рабочий процесс

1.  **Receive Goal (Получение цели):** Агенту дается сложная задача.
2.  **Think/Reason (Думать/Рассуждать):** Агент генерирует внутреннюю мысль, например: *"Чтобы ответить на это, мне сначала нужно найти часть информации X."*
3.  **Act (Действовать):** Основываясь на своей мысли, агент выполняет действие, обычно вызывая инструмент (например, `search_api('X')`).
4.  **Observe (Наблюдать):** Агент получает результат от инструмента.
5.  **Repeat (Повторить):** Агент включает наблюдение в свой контекст и возвращается к шагу 2, генерируя новую мысль (например, *"Окей, теперь, когда у меня есть X, мне нужно использовать его, чтобы найти Y."*). Этот цикл продолжается до тех пор, пока общая цель не будет достигнута.

### Когда использовать / Применение
*   **Multi-hop Question Answering (Многоходовые ответы на вопросы):** Когда ответ на вопрос требует поиска нескольких частей информации последовательно (например, "Кто является генеральным директором компании, которая производит iPhone?").
*   **Web Navigation & Research (Веб-навигация и исследования):** Агент может искать отправную точку, читать результаты, а затем принимать решение о новом поисковом запросе на основе того, что он узнал.
*   **Interactive Workflows (Интерактивные рабочие процессы):** Любая задача, где среда динамична и полный путь к решению не может быть известен заранее.

### Сильные и слабые стороны
*   **Сильные стороны:**
    *   **Adaptive & Dynamic (Адаптивность и динамичность):** Может корректировать свой план на лету на основе новой информации.
    *   **Handles Complexity (Справляется со сложностью):** Преуспевает в проблемах, требующих цепочки из нескольких зависимых шагов.
*   **Слабые стороны:**
    *   **Higher Latency & Cost (Высокая задержка и стоимость):** Включает несколько последовательных вызовов LLM, что делает его медленнее и дороже, чем подходы single-shot.
    *   **Risk of Loops (Риск циклов):** Плохо направляемый агент может застрять в повторяющихся, непродуктивных циклах мыслей и действий.

## Фаза 0: Основа и Настройка

Мы начнем с нашего стандартного процесса настройки: установка библиотек и конфигурация API ключей для Nebius, LangSmith и нашего инструмента веб-поиска Tavily.

### Шаг 0.1: Установка основных библиотек

**Что мы будем делать:**
Мы установим наш стандартный набор библиотек для этой серии проектов.

In [1]:
# !pip install -q -U langchain-nebius langchain langgraph rich python-dotenv tavily-python

### Шаг 0.2: Импорт библиотек и настройка ключей

**Что мы будем делать:**
Мы импортируем необходимые модули и загрузим наши API ключи из файла `.env`.

**Требуемое действие:** Создайте файл `.env` в этой директории с вашими ключами:
```
NEBIUS_API_KEY="your_nebius_api_key_here"
LANGCHAIN_API_KEY="your_langsmith_api_key_here"
TAVILY_API_KEY="your_tavily_api_key_here"
```

In [ ]:
import os
import json
from typing import List, Annotated, TypedDict, Optional
from dotenv import load_dotenv

# Компоненты LangChain
from langchain_nebius import ChatNebius
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage
from pydantic import BaseModel, Field

# Компоненты LangGraph
from langgraph.graph import StateGraph, END
from langgraph.graph.message import AnyMessage, add_messages
from langgraph.prebuilt import ToolNode, tools_condition

# Для красивого вывода
from rich.console import Console
from rich.markdown import Markdown

from notebook_utils import setup_environment

# --- Настройка API ключей и трассировки ---
console = setup_environment(
    project_name="Agentic Architecture - ReAct (Nebius)",
    required_keys=["NEBIUS_API_KEY", "LANGCHAIN_API_KEY", "TAVILY_API_KEY"]
)

Environment variables loaded and tracing is set up.


## Фаза 1: Базовый подход — Single-Shot Tool User

Чтобы понять, почему ReAct так мощен, мы должны сначала увидеть, что происходит без него. Мы построим "базового" агента, который может использовать инструменты, но только один раз. Он проанализирует запрос пользователя, сделает один вызов инструмента, а затем попытается сформулировать окончательный ответ на основе этой единственной части информации.

### Шаг 1.1: Построение базового агента

**Что мы будем делать:**
Мы определим тот же инструмент и LLM, что и раньше, но мы соединим их в простой, линейный граф. Агент получает один шанс вызвать инструмент, и затем рабочий процесс завершается. Цикла нет.

In [ ]:
from typing import TypedDict

console = Console()

# Определяем состояние для наших графов
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

# Определяем инструмент и LLM
search_tool = TavilySearchResults(max_results=2, name="web_search")
llm = ChatNebius(model="meta-llama/Meta-Llama-3.1-8B-Instruct", temperature=0)
llm_with_tools = llm.bind_tools([search_tool])

# Определяем узел агента для базового агента
def basic_agent_node(state: AgentState):
    console.print("--- BASIC AGENT: Thinking... (БАЗОВЫЙ АГЕНТ: Думает...) ---")
    # Примечание: Мы предоставляем системный промпт, чтобы побудить его ответить сразу после одного вызова инструмента
    system_prompt = "You are a helpful assistant. You have access to a web search tool. Answer the user's question based on the tool's results. You must provide a final answer after one tool call."
    messages = [("system", system_prompt)] + state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

# Определяем базовый, линейный граф
basic_graph_builder = StateGraph(AgentState)
basic_graph_builder.add_node("agent", basic_agent_node)
basic_graph_builder.add_node("tools", ToolNode([search_tool]))

basic_graph_builder.set_entry_point("agent")
# После агента он может пойти только к инструментам, а после инструментов он ОБЯЗАН завершить работу.
basic_graph_builder.add_conditional_edges("agent", tools_condition, {"tools": "tools", "__end__": "__end__"})
basic_graph_builder.add_edge("tools", END)

basic_tool_agent_app = basic_graph_builder.compile()

print("Базовый single-shot агент, использующий инструменты, успешно скомпилирован.")

C:\Users\faree\AppData\Local\Temp\ipykernel_22948\1990219742.py:10: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  search_tool = TavilySearchResults(max_results=2, name="web_search")


Basic single-shot tool-using agent compiled successfully.


### Шаг 1.2: Тестирование базового агента на многошаговой задаче

**Что мы будем делать:**
Теперь мы дадим базовому агенту задачу, которая требует нескольких зависимых шагов для решения. Это обнажит его фундаментальную слабость.

In [ ]:
multi_step_query = "Who is the current CEO of the company that created the sci-fi movie 'Dune', and what was the budget for that company's most recent film?"

console.print(f"[bold yellow]Тестирование БАЗОВОГО агента на многошаговом запросе:[/bold yellow] '{multi_step_query}'\n")

basic_agent_output = basic_tool_agent_app.invoke({"messages": [("user", multi_step_query)]})

console.print("\n--- [bold red]Final Output from Basic Agent (Финальный вывод от базового агента)[/bold red] ---")
console.print(Markdown(basic_agent_output['messages'][-1].content))

Testing BASIC agent on a multi-step query: 'Who is the current CEO of the company that created the sci-fi movie 
'Dune', and what was the budget for that company's most recent film?'

--- BASIC AGENT: Thinking... ---

--- Final Output from Basic Agent ---

[{"title": "Dune (2021 film) - Wikipedia", "url": "https://en.wikipedia.org/wiki/Dune_(2021_film)", "content":     
"Dune is a 2021 American epic space opera film directed and co-produced by Denis Villeneuve, who co-wrote the      
screenplay with Jon Spaihts and Eric Roth.", "score": 0.5622973}, {"title": "Dune: Part Two | Dune Wiki - Fandom", 
"url": "https://dune.fandom.com/wiki/Dune:_Part_Two", "content": "sequel will also be produced by Mary Parent, and 
Cale Boyter, with Tanya Lapointe, Brian Herbert, Byron Merritt, Kim Herbert, Thomas Tull, Jon Spaihts, Richard P.  
Rubinstein, John Harrison, and Herbert W. Gains serving as executive producers and Kevin J. Anderson as creative   
consultant. Legendary CEO Joshua Grode confirmed in April 2019 that they plan to make a sequel, adding that        
"there's a logical place to stop the [first] movie before the book is over". [...] By November 2016, Legendary     
Entertainment had obtained the film and TV rights for the Dune franchise, based on the eponymous 1965 novel by     
Frank Herbert. Vice chair of worldwide production for Legendary Mary Parent began discussing with Denis Villeneuve 
about directing a film adaptation, quickly hiring him after realizing his passion for Dune. By February 2018,      
Villeneuve was confirmed to be hired as director, and intended to adapt the novel as a two-part film series.       
Villeneuve ultimately [...] work with Timothée Chalamet and Zendaya again, while stating Chani will have a bigger  
role in the sequel. Warner Bros. assured Villeneuve a sequel would be greenlit as long as the film performs well on
HBO Max.Just days prior to the first film's release, Warner Bros. CEO Ann Sarnoff stated, "Will we have a sequel to
Dune? If you watch the movie, you see how it ends. I think you pretty much know the answer to that."", "score":    
0.53895956}]

**Обсуждение вывода:**
Как и ожидалось, базовый агент потерпел неудачу. Его единственный вызов инструмента, скорее всего, был поиском всего длинного запроса целиком. Результаты поиска для такого сложного, составного запроса часто беспорядочны и не содержат всех необходимых частей информации в одном месте.

Финальный ответ агента, вероятно, является неполным, неверным или утверждением, что он не может найти информацию. Он не смог разбить проблему на части:
1.  Найти компанию, которая создала 'Dune' (Legendary Entertainment).
2.  Найти генерального директора этой компании (Joshua Grode).
3.  Найти самый последний фильм этой компании и его бюджет.

Эта неудача прекрасно иллюстрирует необходимость более динамичного подхода. Агенту нужен способ **реагировать (react)** на информацию, которую он находит на одном шаге, чтобы информировать следующий.

## Фаза 2: Продвинутый подход — Реализация ReAct

Теперь мы построим настоящего агента ReAct. Главное отличие заключается в структуре графа: мы введем цикл, который позволит агенту многократно думать, действовать и наблюдать.

### Шаг 2.1: Построение графа агента ReAct

**Что мы будем делать:**
Мы определим узлы и критически важную функцию маршрутизатора, которая создает цикл `think -> act`. Ключевым архитектурным изменением является ребро, которое направляет вывод из `tool_node` *обратно* к `agent_node`, позволяя агенту видеть результаты и принимать решение о своем следующем шаге.

In [ ]:
def react_agent_node(state: AgentState):
    console.print("--- REACT AGENT: Thinking... (АГЕНТ REACT: Думает...) ---")
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

# ToolNode тот же, что и раньше
react_tool_node = ToolNode([search_tool])

# Маршрутизатор (router) тоже имеет ту же логику
def react_router(state: AgentState):
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        console.print("--- ROUTER: Decision is to call a tool. (МАРШРУТИЗАТОР: Решение вызвать инструмент.) ---")
        return "tools"
    console.print("--- ROUTER: Decision is to finish. (МАРШРУТИЗАТОР: Решение завершить.) ---")
    return "__end__"

# Теперь мы определяем граф с критически важным циклом
react_graph_builder = StateGraph(AgentState)
react_graph_builder.add_node("agent", react_agent_node)
react_graph_builder.add_node("tools", react_tool_node)

react_graph_builder.set_entry_point("agent")
react_graph_builder.add_conditional_edges("agent", react_router, {"tools": "tools", "__end__": "__end__"})

# В этом и заключается ключевое отличие: ребро идет от инструментов ОБРАТНО к агенту
react_graph_builder.add_edge("tools", "agent")

react_agent_app = react_graph_builder.compile()
print("Агент ReAct успешно скомпилирован с циклом рассуждений.")

ReAct agent compiled successfully with a reasoning loop.


## Фаза 3: Сравнительный анализ (Head-to-Head Comparison)

Теперь мы запустим тот же сложный запрос с нашим новым агентом ReAct и понаблюдаем разницу в его процессе и финальном выводе.

### Шаг 3.1: Тестирование агента ReAct на многошаговой задаче

**Что мы будем делать:**
Мы вызовем агента ReAct с тем же многошаговым запросом и будем стримить вывод, чтобы увидеть его итеративный процесс рассуждений.

In [ ]:
console.print(f"[bold green]Тестирование агента ReAct на том же многошаговом запросе:[/bold green] '{multi_step_query}'\n")

final_react_output = None
for chunk in react_agent_app.stream({"messages": [("user", multi_step_query)]}, stream_mode="values"):
    final_react_output = chunk
    console.print(f"--- [bold purple]Current State (Текущее состояние)[/bold purple] ---")
    chunk['messages'][-1].pretty_print()
    console.print("\n")

console.print("\n--- [bold green]Final Output from ReAct Agent (Финальный вывод от агента ReAct)[/bold green] ---")
console.print(Markdown(final_react_output['messages'][-1].content))

Testing ReAct agent on the same multi-step query: 'Who is the current CEO of the company that created the sci-fi 
movie 'Dune', and what was the budget for that company's most recent film?'

--- Current State ---

================================ Human Message =================================

Who is the current CEO of the company that created the sci-fi movie 'Dune', and what was the budget for that company's most recent film?


--- REACT AGENT: Thinking... ---

--- ROUTER: Decision is to call a tool. ---

...

--- ROUTER: Decision is to finish. ---

--- Current State ---

================================== Ai Message ==================================

The current CEO of the company that created the sci-fi movie 'Dune' is Joshua Grode. However, I couldn't find the budget for the company's most recent film in the search results.


--- Final Output from ReAct Agent ---

The current CEO of the company that created the sci-fi movie 'Dune' is Joshua Grode. However, I couldn't find the  
budget for the company's most recent film in the search results.

**Обсуждение вывода:**
Успех! Трассировка выполнения показывает совершенно другой и гораздо более интеллектуальный процесс. Вы можете видеть пошаговое рассуждение агента:
1.  **Thought 1 (Мысль 1):** Сначала он рассуждает, что ему нужно определить производственную компанию фильма 'Dune'.
2.  **Action 1 (Действие 1):** Он вызывает инструмент `web_search` с запросом вроде "production company for Dune movie".
3.  **Observation 1 (Наблюдение 1):** Он получает результат: "Legendary Entertainment".
4.  **Thought 2 (Мысль 2):** Теперь, используя новую информацию, он рассуждает, что ему нужен генеральный директор Legendary Entertainment.
5.  **Action 2 (Действие 2):** Он вызывает `web_search` снова с запросом вроде "CEO of Legendary Entertainment".
6.  ...и так далее, пока не соберет все необходимые части.
7.  **Synthesis (Синтез):** В конце концов, он собирает все полученные факты в полный и точный ответ.

Это ясно демонстрирует превосходство паттерна ReAct для любой задачи, которая не является простым одношаговым поиском.

## Фаза 4: Количественная оценка

Чтобы формализовать сравнение, мы используем LLM-как-судью (LLM-as-a-Judge) для оценки финальных выводов как от базового агента, так и от агента ReAct по их способности решить задачу.

In [ ]:
class TaskEvaluation(BaseModel):
    """Схема для оценки способности агента выполнить задачу."""
    task_completion_score: int = Field(description="Оценка 1-10: успешно ли агент выполнил все части запроса пользователя.")
    reasoning_quality_score: int = Field(description="Оценка 1-10: логический поток и процесс рассуждения, продемонстрированный агентом.")
    justification: str = Field(description="Краткое обоснование оценок.")

judge_llm = llm.with_structured_output(TaskEvaluation)

def evaluate_agent_output(query: str, agent_output: dict):
    trace = "\n".join([f"{m.type}: {m.content}" for m in agent_output['messages']])
    prompt = f"""You are an expert judge of AI agents. Evaluate the following agent's performance on the given task on a scale of 1-10. A score of 10 means the task was completed perfectly. A score of 1 means complete failure.
    
    **User's Task:**
    {query}
    
    **Full Agent Conversation Trace:**
    ```
    {trace}
    ```
    """
    return judge_llm.invoke(prompt)

console.print("--- Evaluating Basic Agent's Output (Оценка вывода базового агента) ---")
basic_agent_evaluation = evaluate_agent_output(multi_step_query, basic_agent_output)
console.print(basic_agent_evaluation.model_dump())

console.print("\n--- Evaluating ReAct Agent's Output (Оценка вывода агента ReAct) ---")
react_agent_evaluation = evaluate_agent_output(multi_step_query, final_react_output)
console.print(react_agent_evaluation.model_dump())

--- Evaluating Basic Agent's Output ---

{
    'task_completion_score': 2,
    'reasoning_quality_score': 4,
    'justification': "The agent provided some relevant information about the movie 'Dune' and its sequel, but 
failed to answer the user's question directly. It did not provide the name of the current CEO of the company that 
created the movie, nor the budget for the company's most recent film. The agent's response was somewhat relevant, 
but lacked focus and clarity."
}

--- Evaluating ReAct Agent's Output ---

{
    'task_completion_score': 6,
    'reasoning_quality_score': 8,
    'justification': "The agent correctly identified the current CEO of the company that created the sci-fi movie 
'Dune' as Joshua Grode. However, it failed to find the budget for the company's most recent film, which is a 
crucial part of the task. The agent's reasoning quality is high as it provided relevant information from Wikipedia 
and IMDb, but it lacked the ability to extract the required information from the search results."
}

**Обсуждение вывода:**
Количественные оценки от LLM-судьи делают разницу кристально ясной.
- **Базовый агент (Basic Agent)** получил очень низкий `task_completion_score`, потому что он не смог собрать всю необходимую информацию. Его оценка качества рассуждений `reasoning_quality_score` также низкая, потому что его процесс был ошибочным и неполным.
- **Агент ReAct**, напротив, получил почти идеальные оценки. Судья признал, что его итеративный процесс позволил ему успешно выполнить все части сложной задачи.

Это очное сравнение и оценка предоставляют окончательное доказательство ценности архитектуры ReAct. Это ключ, который открывает агенту возможность решать сложные, многоходовые проблемы, требующие динамической адаптации.

## Заключение

В этом ноутбуке мы не только реализовали архитектуру **ReAct**, но и продемонстрировали ее явное превосходство над более простым подходом single-shot. Создав рабочий процесс, который позволяет агенту зацикливать цикл рассуждения и действий, мы позволили ему решать сложные, многошаговые проблемы, которые в противном случае были бы неразрешимы.

Способность наблюдать результат действия и использовать эту информацию для информирования следующего шага является фундаментальным компонентом интеллектуального поведения. Паттерн ReAct предоставляет простой, но глубоко эффективный способ встроить эту способность в наших ИИ-агентов, делая их более мощными, адаптивными и полезными для реальных задач.